<a href="https://colab.research.google.com/github/Findoflad/kursach_bot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.optimizers import gradient_descent_v2 
import random
from keras.models import load_model

# загружаем файл для подготовки
words = []
classes = []
documents = []
ignore_words = ['?','!']
data_file = open("/content/sample_data/Train_Bot.json").read()
intents = json.loads(data_file)

# подготавливаем данные для обработки, в частности токенизируем их
# и разбиваем на слова, документы, классы
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    # добавляем документы к корпусу
    documents.append((w, intent['tag']))

    if intent['tag'] not in classes:
      classes.append(intent['tag'])

# теперь лемматизация
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

# применяя сериализацию сохраняем полученные объекты для дальнейшего использования
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# создаем данные для обучения
training = []

# объявляем пустой массив для вывода
output_empty = [0] * len(classes)

# создаем модель мешка слов для каждого предложения
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

# создаем списки для тренировки и тестов
train_x = list(training[:,0])
train_y = list(training[:,1])

# последовательная модель для предсказания ответов
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# компилируем модель
sgd = gradient_descent_v2.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
# сохраняем и загружаем для дальнейшего использования
#model.save('chatbot.h5', hist)
model = load_model('/content/chatbot.h5')

# теперь пользовательский ввод
def clean_up_sentence(sentence):
  # токенизируем предложение
  sentence_words = nltk.word_tokenize(sentence)
  # и приводим каждое слово к первоначальной форме
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

# модель мешка слов для введенного предложения
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)

  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:

        bag[i] = 1
        if show_details:
          print("found in bag: %s" % w)
  return(np.array(bag))

# функция для прогноза ответа
def predict_class(sentence, model):
  p = bow(sentence, words, show_details=False)
  res = model.predict(np.array([p]))[0]
  error = 0.25
  results = [[i,r] for i,r in enumerate(res) if r>error]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []

  for r in results:
    return_list.append({"intent": classes[r[0]], "probability:": str(r[1])})
  return return_list

# функция получения ответа от модели
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if(i['tag']==tag):
      result = random.choice(i['responses'])
      break
  return result

# функция для прогноза класса и вывода соответствующего ответа
def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

# запус бота
def start_chat():
  print("Bot: Hello Master. \n\n")
  while True:
    inp = str(input()).lower()
    if inp.lower()=="end":
      break
    if inp.lower()=='' or inp.lower()=='*':
      print("Please rewrite your phrase")
      print("_"*50)
    else:
      print(f"Bot: {chatbot_response(inp)}"+'\n')
      print("_"*50)

start_chat()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Bot: Hello Master. 


end
